In [10]:
from utils import read_lines
from collections import defaultdict
from collections import deque


def calc(rand1, rand2, op):
    match op:
        case "AND":
            return rand1 & rand2
        case "OR":
            return rand1 | rand2
        case "LSHIFT":
            return rand1 << rand2
        case "RSHIFT":
            return rand1 >> rand2
        case _:
            raise ValueError(f"invalid op {op}")


def get_value(values, k):
    if isinstance(k, int):
        return k
    else:
        return values[k]


def parse_input(input_file):
    indegree = defaultdict(int)
    downstream = defaultdict(list)
    instructions = {}
    for line in read_lines(input_file):
        left, right = line.split(" -> ")
        parts = left.split(" ")
        if len(parts) == 1:
            try:
                v = int(parts[0])
            except Exception:
                downstream[parts[0]].append(right)
                indegree[right] = 1
                instructions[right] = [left]
            else:
                indegree[right] = 0
                instructions[right] = [v]
        elif parts[0] == "NOT":
            try:
                v = int(parts[1])

            except Exception:
                downstream[parts[1]].append(right)
                indegree[right] = 1
                instructions[right] = ["NOT", parts[1]]

            else:
                instructions[right] = ["NOT", v]
                indegree[right] = 0
        else:
            v1, v2 = None, None
            try:
                v1 = int(parts[0])
            except Exception:
                indegree[right] += 1
                downstream[parts[0]].append(right)
            try:
                v2 = int(parts[2])
            except Exception:
                indegree[right] += 1
                downstream[parts[2]].append(right)
            if v1 and v2:
                instructions[right] = [parts[1], v1, v2]
            elif v1:
                instructions[right] = [parts[1], v1, parts[2]]
            elif v2:
                instructions[right] = [parts[1], parts[0], v2]
            else:
                instructions[right] = [parts[1], parts[0], parts[2]]
    return indegree, downstream, instructions


def solve(indegree, downstream, instructions):
    values = {}
    q = deque()
    for k, v in indegree.items():
        if v == 0:
            q.append(k)
    while q:
        k = q.popleft()
        instruction = instructions[k]
        try:
            if len(instruction) == 1:
                values[k] = get_value(values, instruction[0])
            elif instruction[0] == "NOT":
                values[k] = ~get_value(values, instruction[1])
            else:
                values[k] = calc(
                    get_value(values, instruction[1]),
                    get_value(values, instruction[2]),
                    instruction[0],
                )
            for down in downstream[k]:
                indegree[down] -= 1
                if indegree[down] == 0:
                    q.append(down)
        except Exception:
            print(k, instruction)
            raise
    return values


def part1(input_file):
    indegree, downstream, instructions = parse_input(input_file)

    values = solve(indegree, downstream, instructions)
    return values["a"]


def part2(input_file):
    indegree, downstream, instructions = parse_input(input_file)

    values = solve(indegree.copy(), downstream, instructions)

    a = values["a"]
    indegree["b"] = 0
    instructions["b"] = [a]

    return solve(indegree, downstream, instructions)["a"]

In [11]:
part1('inputs/day7.txt')

16076

In [12]:
part2('inputs/day7.txt')

2797